In [15]:
# Plotting
from matplotlib import pyplot as plt, figure
from matplotlib.colors import LogNorm
import matplotlib.patches as mpatches

import netCDF4 as nc
from progressbar import progressbar

import cartopy.feature as cfeature
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

from plotly import express as px
from sklearn.metrics import mean_squared_error

from tqdm import tqdm
tqdm.pandas()

import pandas as pd

In [16]:
def get_smap(path: str, printing=False):

    ds = nc.Dataset(path)
    sm = ds['Soil_Moisture_Retrieval_Data_AM']

    latitudes = []
    longitudes = []
    moistures = []
    times = []
    qfs = []
    landcover_01 = []
    landcover_02 = []
    landcover_03 = []
    roughness = []
    surface_temp = []
    vo = []
    veg_wat_cont = []

    for lat in tqdm(range(len(sm['latitude']))):
        for long in range(len(sm['longitude'][lat])):
            latitudes.append(sm['latitude'][lat][long])
            longitudes.append(sm['longitude'][lat][long])
            moistures.append(sm['soil_moisture'][lat][long])
            times.append(sm['tb_time_utc'][lat][long])
            qfs.append(sm['retrieval_qual_flag'][lat][long])
            landcover_01.append(sm['landcover_class.Bands_01'][lat][long])
            landcover_02.append(sm['landcover_class.Bands_02'][lat][long])
            landcover_03.append(sm['landcover_class.Bands_03'][lat][long])
            roughness.append(sm['roughness_coefficient'][lat][long])
            surface_temp.append(sm['surface_temperature'][lat][long])
            vo.append(sm['vegetation_opacity'][lat][long])
            veg_wat_cont.append(sm['vegetation_water_content'][lat][long])        

    df = pd.DataFrame.from_dict({'lat': latitudes, 'long': longitudes, 'time': times, 'smap_sm': moistures,
                                 'retrieval_qfs': qfs, 'surface_roughness': roughness,
                                 'surface_temp': surface_temp, 'vegetation_opacity': vo,
                                 'vegetation_water_content': veg_wat_cont, 'landcover_class_01': landcover_01,
                                 'landcover_class_02': landcover_02, 'landcover_class_03': landcover_03})

    # Filter out missing values
    smap_df = df[df['smap_sm'] != -9999.0]

    if len(smap_df) > 0 and printing:
        print('Number of missing values:', len(df) - len(smap_df))
        print('Number of data points with value:', len(smap_df))
        index = list(smap_df['smap_sm']).index(max(list(smap_df['smap_sm'])))
        print("Peak SM value:", list(smap_df['smap_sm'])[index])
        print("Peak SM value at: (" + str(list(smap_df['lat'])[index]) + ", " + str(list(smap_df['long'])[index]) + ")")

    return smap_df


NameError: name 'latitudes' is not defined

In [14]:
rootdir = '/Users/madsrindal/Desktop/5000003164051/240798506/SMAP_L3_SM_P_20200101_R18290_003_HEGOUT.nc'
full_df = get_smap(rootdir)

100%|██████████| 406/406 [30:51<00:00,  4.56s/it] 


NameError: name 'pd' is not defined

In [ ]:
"""df = full_df.append(full_df2, ignore_index=True)
df = df.append(full_df3, ignore_index=True)
df = df.append(full_df4, ignore_index=True)
df = df.append(full_df5, ignore_index=True)
df = df.append(full_df6, ignore_index=True)"""

In [ ]:
def filter_smap_qfs(smap_df):
    return smap_df.loc[(smap_df['retrieval_qfs'] == 0) | (smap_df['retrieval_qfs'] == 8)]

In [ ]:
location_df = filter_smap_qfs(df)

In [ ]:
print('Lat Min: ', location_df['lat'].min())
print('Lat Max: ', location_df['lat'].max())

print('Lon Min: ', location_df['long'].min())
print('Lon Max: ', location_df['long'].max())

print('Vegetation Opacity Min: ', location_df['vegetation_opacity'].min())
print('Vegetation Opacity Max: ', location_df['vegetation_opacity'].max())

print('Surface Roughness Min: ', location_df['surface_roughness'].min())
print('Surface Roughness Max: ', location_df['surface_roughness'].max())

In [ ]:
def get_plot_ticks(lat_values, long_values):
    min_lat = min(lat_values)
    max_lat = max(lat_values)
    min_long = min(long_values)
    max_long = max(long_values)

    lat_step_size = (max_lat - min_lat) / 3
    long_step_size = (max_long - min_long) / 3

    long_list = [min_long, min_long + long_step_size, min_long + 2 * long_step_size, max_long]
    lat_list = [min_lat, min_lat + lat_step_size, min_lat + 2 * lat_step_size, max_lat]

    # Rounding to two decimals
    long_list = [round(num, 2) for num in long_list]
    lat_list = [round(num, 2) for num in lat_list]

    return lat_list, long_list

In [ ]:
# Set Desired Locations
n1 = -7
w1 = 23
s1 = -12
e1 = 28

n2 = -5
w2 = -42
s2 = -10
e2 = -37

n3 = -22
w3 = 117
s3 = -27
e3 = 122

n4 = 24
w4 = 80
s4 = 19
e4 = 85

n5 = 31
w5 = 59
s5 = 26
e5 = 64

In [ ]:
# PLOT GLOBAL SURFACE ROUGHNESS

fig = plt.figure(figsize=(18, 6))

# Settings for the plot
ax = plt.axes(projection=ccrs.PlateCarree())
sc = plt.scatter(location_df['long'], location_df['lat'], c=list(location_df['surface_roughness']), s=1, cmap='Spectral', zorder=1)
bar = plt.colorbar(sc)
#bar.ax.set_title('SR [dB]')

ax.set_extent([-140, 160, 40, -40], crs=crs.PlateCarree()) ## Set overall area

ax.add_patch(mpatches.Rectangle(xy=[w1, n1], width=(e1-w1), height=(s1-n1), facecolor='red', alpha=0.5, transform=ccrs.PlateCarree()))
ax.add_patch(mpatches.Rectangle(xy=[w2, n2], width=(e2-w2), height=(s2-n2), facecolor='red', alpha=0.5, transform=ccrs.PlateCarree()))
ax.add_patch(mpatches.Rectangle(xy=[w3, n3], width=(e3-w3), height=(s3-n3), facecolor='red', alpha=0.5, transform=ccrs.PlateCarree()))
ax.add_patch(mpatches.Rectangle(xy=[w4, n4], width=(e4-w4), height=(s4-n4), facecolor='red', alpha=0.5, transform=ccrs.PlateCarree()))
ax.add_patch(mpatches.Rectangle(xy=[w5, n5], width=(e5-w5), height=(s5-n5), facecolor='red', alpha=0.5, transform=ccrs.PlateCarree()))

lat_list, long_list = get_plot_ticks([40, -40], [-140, 160])
ax.set_xticks(long_list, crs=ccrs.PlateCarree())
ax.set_yticks(lat_list, crs=ccrs.PlateCarree())
ax.xaxis.set_major_formatter(LongitudeFormatter())
ax.yaxis.set_major_formatter(LatitudeFormatter())

plt.title('Global Surface Rougness', fontsize=18)
plt.xlabel('Longitude', fontsize=12)
plt.ylabel('Latitude', fontsize=12)

# plt.savefig('/Users/madsrindal/Desktop/Plots/AreaOfInterestDemonstration.png')

plt.show()


In [ ]:
# PLOT GLOBAL VEGETATION OPACITY

fig = plt.figure(figsize=(18, 6))

# Settings for the plot
ax = plt.axes(projection=ccrs.PlateCarree())
sc = plt.scatter(location_df['long'], location_df['lat'], c=list(location_df['vegetation_opacity']), s=1, cmap='Spectral', zorder=1)
bar = plt.colorbar(sc)
#bar.ax.set_title('SR [dB]')

ax.set_extent([-140, 160, 40, -40], crs=crs.PlateCarree()) ## Set overall area

ax.add_patch(mpatches.Rectangle(xy=[w1, n1], width=(e1-w1), height=(s1-n1), facecolor='red', alpha=0.5, transform=ccrs.PlateCarree()))
ax.add_patch(mpatches.Rectangle(xy=[w2, n2], width=(e2-w2), height=(s2-n2), facecolor='blue', alpha=0.5, transform=ccrs.PlateCarree()))
ax.add_patch(mpatches.Rectangle(xy=[w3, n3], width=(e3-w3), height=(s3-n3), facecolor='blue', alpha=0.5, transform=ccrs.PlateCarree()))
ax.add_patch(mpatches.Rectangle(xy=[w4, n4], width=(e4-w4), height=(s4-n4), facecolor='blue', alpha=0.5, transform=ccrs.PlateCarree()))
ax.add_patch(mpatches.Rectangle(xy=[w5, n5], width=(e5-w5), height=(s5-n5), facecolor='blue', alpha=0.5, transform=ccrs.PlateCarree()))

lat_list, long_list = get_plot_ticks([40, -40], [-140, 160])
ax.set_xticks(long_list, crs=ccrs.PlateCarree())
ax.set_yticks(lat_list, crs=ccrs.PlateCarree())
ax.xaxis.set_major_formatter(LongitudeFormatter())
ax.yaxis.set_major_formatter(LatitudeFormatter())

plt.title('Global Vegetation Opacity', fontsize=18)
plt.xlabel('Longitude', fontsize=12)
plt.ylabel('Latitude', fontsize=12)

# plt.savefig('/Users/madsrindal/Desktop/Plots/AreaOfInterestDemonstration.png')

plt.show()


In [ ]:
# PLOT DEMONSTRATION OF DESIRED LOCATION

# SET MAP AREA
north = 50
west = -20
south = -50
east = 80

# SET DESIRED LOCATION
n = n1
w = w1
s = s1
e = e1

fig = plt.figure(figsize=(8,8))

ax = fig.add_subplot(1,1,1, projection=crs.PlateCarree())

ax.stock_img()
ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle='-', alpha=.5)
ax.set_extent([west, east, north, south], crs=crs.PlateCarree()) ## Set overall area

lat_list, long_list = get_plot_ticks([north, south], [west, east])
ax.set_xticks(long_list, crs=ccrs.PlateCarree())
ax.set_yticks(lat_list, crs=ccrs.PlateCarree())
ax.xaxis.set_major_formatter(LongitudeFormatter())
ax.yaxis.set_major_formatter(LatitudeFormatter())

ax.add_patch(mpatches.Rectangle(xy=[w, n], width=(e-w), height=(s-n), facecolor='red', alpha=0.4, transform=ccrs.PlateCarree()))

plt.text((e+1), n, '(' + str(e) + ',' + str(n) + ')', horizontalalignment='left', transform=ccrs.Geodetic(), fontsize=9, color='black', weight='bold')
plt.text((w-1), s, '(' + str(w) + ',' + str(s) + ')', horizontalalignment='right', transform=ccrs.Geodetic(), fontsize=9, color='black', weight='bold')

plt.title('Area of Interest for Demonstration', fontsize=18)
plt.xlabel('Longitude', fontsize=12)
plt.ylabel('Latitude', fontsize=12)

plt.plot(w, n, color='red', marker='o', alpha=0.4, transform=ccrs.PlateCarree())
plt.plot(w, s, color='red', marker='o', alpha=0.4, transform=ccrs.PlateCarree())
plt.plot(e, n, color='red', marker='o', alpha=0.4, transform=ccrs.PlateCarree())
plt.plot(e, s, color='red', marker='o', alpha=0.4, transform=ccrs.PlateCarree())

#plt.savefig('/Users/madsrindal/Desktop/Plots/AreaOfInterestAfrica.png')

plt.show()
